In [ ]:
!pip install beautifulsoup4 
!pip install matplotlib
!pip install requests
!pip install nltk
!pip install pandas
!pip install numpy as np


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import gspread
from datetime import datetime
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import json
import nltk
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()


In [ ]:
# To make it easier to update and share this notebook we will be entering sensitive keys are runtime using nbfire.  T
# This cell is a placeholder for these values.  This is why the cell has a 'parameters' tag added.  For more
#information please read https://papermill.readthedocs.io/en/latest/usage-parameterize.html
credentials = ""


In [ ]:
#We need to handle both Google API credentials added as a string entered as a parameter on NBFire and as a dict
# entered directly in the cell above
if isinstance(credentials, dict):
    sa = gspread.service_account_from_dict(credentials)
elif isinstance(credentials, str):
    creds = json.loads(credentials)
    sa = gspread.service_account_from_dict(creds)

In [ ]:
sheet = sa.open_by_key('1NhgBt6ipU_9A0vekRfOYspl1Z-GKRU12gY7R19Zjvag')
daily_guardian_datasheet = sheet.worksheet("Daily_Guardian_Sentiments")
daily_telegraph_datasheet = sheet.worksheet("Daily_Telegraph_Sentiments")
mean_datasheet=sheet.worksheet("Daily_Mean_Sentiments")

In [ ]:
guardian_url = 'https://www.guardian.co.uk'
telegraph_url = 'https://telegraph.co.uk'
current_datetime = datetime.now()
now = current_datetime.strftime("%Y-%m-%d")

In [ ]:
def get_guardian_titles(url):
    article = requests.get(url)
    soup = bs(article.text, 'html.parser')
    headlines = soup.find('body').find_all('h3')
    for x in headlines[:15]:
        try:
            if x.div.text != 'Live':
                title= x.div.text
                description= x.span.text
                complete_text = title + " - " + description
                scores=analyzer.polarity_scores(complete_text)
                new_entry=[now,complete_text,scores["compound"]]
                print(new_entry)
                daily_guardian_datasheet.append_row(new_entry)
        except: print('error')

In [ ]:
def get_telegraph_titles(url):
    article = requests.get(url)
    soup = bs(article.text, 'html.parser')
    headlines = soup.find_all("span", {"class": ""})
    for x in headlines[4:19]:
        try:
            complete_text = x.text
            scores=analyzer.polarity_scores(complete_text)
            new_entry=[now,complete_text,scores["compound"]]
            print(new_entry)
            daily_telegraph_datasheet.append_row(new_entry)   
        except: print('error')     

In [ ]:
get_guardian_titles(guardian_url)


In [ ]:
get_telegraph_titles(telegraph_url)

In [ ]:
guardian_values = daily_guardian_datasheet.col_values(3)
guardian_sentiments=guardian_values[-15:]

In [ ]:
telegraph_values = daily_telegraph_datasheet.col_values(3)
telegraph_sentiments=telegraph_values[-15:]

In [ ]:
x = np.array(guardian_sentiments)
y = x.astype(float)
mean = np.mean(y)

new_mean_entry=[now, mean]
mean_datasheet.append_row(new_mean_entry)


In [ ]:
x = np.array(telegraph_sentiments)
y = x.astype(float)
mean = np.mean(y)

last_row = len(mean_datasheet.get_all_values())
mean_datasheet.update_cell(last_row, 3, mean)


In [ ]:

df = pd.DataFrame(mean_datasheet.get_all_records())

In [ ]:
ax = df.plot(x="Date", y="Mean_Guardian_Sentiment", figsize=(10, 10))
df.plot('Date', 'Mean_Telegraph_Sentiment', secondary_y=True, ax=ax)


In [ ]:
df